<a href="https://colab.research.google.com/github/MedAljane/Summer_Internship_Project_files/blob/main/Acctualite_touristique_sanitaires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-splitter
!pip install transformers
!pip install SentencePiece
!pip install cdifflib
!pip install deep-translator

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sns
import urllib.request
import json
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
from deep_translator import GoogleTranslator
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import datetime
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Cette fonction importe un modèle pré-entraîné à partir d'un dépôt GitHub créé par "tuner007"
# Ainsi le script doit s'exécuter sur un GPU pour des performances accélérées

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

# TransPrasing Function

In [ ]:
def transPhrasing(text):
    splitter = SentenceSplitter(language='fr')
    sentence_list = splitter.split(text)

    translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
    paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
    cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
    sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
    paraphrased_result = [' '.join(x for x in sentences_to_french)]
    paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')

    return paraphrased_result_cleaned

# Departement Paris

## Actualites

In [ ]:
req = requests.get('https://actu.fr/ile-de-france/paris_75056')
soup = BeautifulSoup(req.content , 'html.parser')

links = []
artPrevLarge = soup.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links.append(artPrevLarge)

artPrevSmall= soup.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall:
    links.append(e.find('a')['href'])

artPrevMedium= soup.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium:
    links.append(e.find('a')['href'])
Publish_Dates= []
Articles_Content = []
liensActualiteParis = []
imagesActualiteParis = []
titreActualiteParis = []

l=links[0:6]
for link in l:
    r=requests.get(link)
    soup=BeautifulSoup(r.content , 'html.parser')

        #Get the link of the article
    liensActualiteParis.append(link)

        #Get the title of the article
    titreActualiteParis.append(transPhrasing(soup.find('div', {'class': 'ac-article-main '}).find('h1').text.strip()))

        #Get the preview image of the article
    imagesActualiteParis.append(soup.find('article', {'class': 'js-article-inner'}).find('img')['src'])

        #Get the paragraphs just under the div
    t = soup.find('div', class_='ac-article-content')
    children = t.findChildren("p" , recursive=False)
    art_content= ""
    for e in children: 
        if e.name == 'p':
            art_content += e.text
    art_content = art_content.replace(u'\xa0', u' ')

        #Get the publishing date
    pub_date = GoogleTranslator(target='fr').translate(datetime.datetime.fromisoformat(soup.find('div', class_='ac-article-date').find('time')['datetime']).strftime("%B %d, %Y at %H:%M"))
    
    Publish_Dates.append(pub_date)
    Articles_Content.append(art_content)


In [ ]:
paris_tou=[]
for i in Articles_Content:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  paris_tou.append(paraphrased_result_cleaned)

In [ ]:
Articles = {
        'dates': Publish_Dates,
        'image article': imagesActualiteParis,
        'actualites' : paris_tou,
        'liensActualite': liensActualiteParis,
        'titreActualite': titreActualiteParis
    } 
paris_dft = pd.DataFrame(Articles)
paris_dft

In [ ]:
paris_t=paris_dft.transpose()


In [ ]:
paris_t

In [ ]:
#département de paris ( Coronavirus )
p=requests.get('https://www.sortiraparis.com/actualites/coronavirus')
soup=BeautifulSoup(p.content , 'html.parser')
liens=[]
for i in soup.findAll('div',{'class':'item'}):
    l=i.find('a')
    liens.append("https://www.sortiraparis.com"+l['href'])
article1=[]
article2=[]
dates_cor=[]
for li in liens :
  p=requests.get(li)
  soup=BeautifulSoup(p.content , 'html.parser')
  for i in soup.findAll('div',{'class':'published'}):
    dates_cor.append(i.text)
  try :
    for i in soup.findAll('div',{'class':'abstract'})[0]:
      article1.append(i)
  except:
    pass
  for j in soup.findAll('div',{'class':'article'}):
    article2.append(j.text)
test1=[]
for i in range(len(article1)):
  test1.append(article1[i])
liste1=[]
for i in test1 :
  i = re.sub(r'\[[0-9]*\]', ' ', i)
  i = re.sub(r'\s+', ' ', i)
  liste1.append(i)

test2=[]
for i in range(len(article2)):
  test2.append(article2[i])
liste2=[]
for i in test2 :
  i = re.sub(r'\[[0-9]*\]', ' ', i)
  i = re.sub(r'\s+', ' ', i)
  liste2.append(i)
list_cor = ['{} {}'.format(x,y) for x,y in zip(liste1,liste2)]
d=[]
for i in dates_cor :
  i = re.sub(r'\[[0-9]*\]', ' ', i)
  i = re.sub(r'\s+', ' ', i)
  d.append(i)
txt2=""
for i in d :
    txt2 += i 
    t2=txt2.split('Publié')
liste=[]
for i in t2[1:len(t2)] :
  liste.append(i)
date_c=[]
for i in liste:
  date_c.append(i[0:60])


#Recuperations des images
imagesActualiteDepParis = ['https://www.sortiraparis.com'+BeautifulSoup(requests.get(li).content , 'html.parser').find('div', {'id': 'article-carousel'}).find('a')['href'] for li in liens[:6]]
liensActualiteDepParis = [li for li in liens[:6]]

In [ ]:
paris_cor=[]
for i in list_cor[:6]:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  paris_cor.append(paraphrased_result_cleaned)

In [ ]:
dict = {'date': date_c[:6],
        'image actualite': imagesActualiteDepParis[:6],
        'actualite' : paris_cor[:6],
        'liensActualite': liensActualiteDepParis
        } 
paris_df1 = pd.DataFrame(dict)
paris_df1

In [ ]:
paris_c=paris_df1.transpose()


In [ ]:
# seine-et-marne
p=requests.get('https://www.seine-et-marne.fr/fr/actualites')
soup=BeautifulSoup(p.content , 'html.parser')
liens1=[]
dates=[]

for i in soup.findAll('article',{'class':'ribbon-item link-context__context -news node--promoted node--view-mode-teaser'}):
  liens1.append('https://www.seine-et-marne.fr'+i.a['href'])
for i in soup.findAll('span',{'class':'date__item'}):
  dates.append(i.text)
artic=[]
for i in liens1 :
  p=requests.get(i)
  soup=BeautifulSoup(p.content , 'html.parser')
  for i in soup.findAll('section',{'class':'site-content'}):
    artic.append(i.text)
test=[]
for i in range(len(artic)):
  test.append(artic[i])
liste=[]
for i in test :
  i = re.sub(r'\[[0-9]*\]', ' ', i)
  i = re.sub(r'\s+', ' ', i)
  liste.append(i)
dd=dates[0:6]
ls=liste[0:6]
#Recuperation images des articles:
imagesActualiteSeM = [BeautifulSoup(requests.get(li).content, 'html.parser').find('div', {'class': 'heading__group'}).find('img')['src'] for li in liens1[:6]]
liensActualiteSeM = [li for li in liens1[:6]]

In [ ]:
seine_tour=[]
for i in ls:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  seine_tour.append(paraphrased_result_cleaned)

In [ ]:
dict = {'date': dd,
        'image actualite': imagesActualiteSeM,
        'acctualite' : seine_tour,
        'liensActualite': liensActualiteSeM
        } 
seine_df1 = pd.DataFrame(dict)
seine_df1

In [ ]:
seine_t=seine_df1.transpose()


In [ ]:
# seine-et-marne (coronavirus)

req_SeM = requests.get('https://actu.fr/la-republique-de-seine-et-marne/societe/coronavirus')
soup_SeM = BeautifulSoup(req_SeM.content , 'html.parser')

links_SeM = []
artPrevLarge_SeM = soup_SeM.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links_SeM.append(artPrevLarge_SeM)

artPrevSmall_SeM = soup_SeM.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall_SeM:
    links_SeM.append(e.find('a')['href'])

artPrevMedium_SeM= soup_SeM.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium_SeM:
    links_SeM.append(e.find('a')['href'])

Publish_Dates_SeM = []
Articles_Content_SeM = []
ls=links_SeM[0:6]
for link in ls:
    r=requests.get(link)
    soup_SeM=BeautifulSoup(r.content , 'html.parser')
    t = soup_SeM.find('div', class_='ac-article-content')

    children = t.findChildren("p" , recursive=False)
    art_content_SeM = ""
    for e in children:
        if e.name == 'p':
            art_content_SeM += e.text
    art_content_SeM = art_content_SeM.replace(u'\xa0', u' ')

    pub_date_SeM = soup_SeM.find('div', class_='ac-article-date').find('time')['datetime']
    
    Publish_Dates_SeM.append(pub_date_SeM)
    Articles_Content_SeM.append(art_content_SeM)

#Recuperation images des articles:
imagesActualiteSeMCor = [BeautifulSoup(requests.get(li).content, 'html.parser').find('article', {'class': 'js-article-inner'}).find('img')['src'] for li in ls]
liensActualiteSeMCor = [li for li in ls]

In [ ]:
seine_cor=[]
for i in Articles_Content_SeM:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  seine_cor.append(paraphrased_result_cleaned)

In [ ]:
date_sen=[]
try:
  for i in Publish_Dates_SeM:
    date_sen.append(i.replace('T','  '))
except:
  date_sen=Publish_Dates_SeM
date_s=[ r.strip().strip().split('+')[0] for r in date_sen ]
Articles_SeM = {'date': date_s,
                'image actualite': imagesActualiteSeMCor,
                'actualite' : seine_cor,
                'liensActualite': liensActualiteSeMCor
        } 
SeM_df = pd.DataFrame(Articles_SeM)
SeM_df

In [ ]:
seine_c=SeM_df.transpose()

In [ ]:
#département de yvelines 
req = requests.get('https://actu.fr/78actu/')
soup = BeautifulSoup(req.content , 'html.parser')

links_yveline = []
artPrevLarge_yveline = soup.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links_yveline.append(artPrevLarge_yveline)

artPrevSmall_yveline = soup.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall_yveline:
    links_yveline.append(e.find('a')['href'])

artPrevMedium_yveline= soup.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium_yveline:
    links_yveline.append(e.find('a')['href'])
Publish_Dates_yveline = []
Articles_Content_yveline = []
links=links_yveline[0:6]
for link in links:
    r=requests.get(link)
    soup=BeautifulSoup(r.content , 'html.parser')
    t = soup.find('div', class_='ac-article-content')

    #Get the paragraphs just under the div
    children = t.findChildren("p" , recursive=False)
    art_content_yveline= ""
    for e in children:
        if e.name == 'p':
            art_content_yveline += e.text
    art_content_yveline = art_content_yveline.replace(u'\xa0', u' ')

    #Get the publishing time
    pub_date_yveline= soup.find('div', class_='ac-article-date').find('time')['datetime']
    
    Publish_Dates_yveline.append(pub_date_yveline)
    Articles_Content_yveline.append(art_content_yveline)
    
dates=[]
try:
  for i in Publish_Dates_yveline:
    dates.append(i.replace('T','  '))
except:
  dates=Publish_Dates_yveline
date_y=[ r.strip().strip().split('+')[0] for r in dates ]

#Recuperation images des articles:
imagesActualiteYveline = [BeautifulSoup(requests.get(li).content, 'html.parser').find('article', {'class': 'js-article-inner'}).find('img')['src'] for li in links_yveline[:6]]
liensActualiteYveline = [li for li in links_yveline[:6]]

In [ ]:
yveline_tou=[]
for i in Articles_Content_yveline:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  yveline_tou.append(paraphrased_result_cleaned)

In [ ]:
Articles_yveline= {
        'dates': date_y,
        'image actualite': imagesActualiteYveline,
        'actualites' : yveline_tou,
        'liensActualite': liensActualiteYveline
        } 
yveline_dft = pd.DataFrame(Articles_yveline)
yveline_dft

In [ ]:
yveline_t=yveline_dft.transpose()

### département de yvelines (coronavirus)

In [ ]:
# Recuperation de liens des articles
req = requests.get('https://actu.fr/78actu/societe/coronavirus')
soup = BeautifulSoup(req.content , 'html.parser')

links_yveline = []
artPrevLarge_yveline = soup.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links_yveline.append(artPrevLarge_yveline)

artPrevSmall_yveline = soup.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall_yveline:
    links_yveline.append(e.find('a')['href'])

artPrevMedium_yveline= soup.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium_yveline:
    links_yveline.append(e.find('a')['href'])

# Recuperation de contenus des articles
Publish_Dates_yveline = []
Articles_Content_yveline = []

for link in links_yveline[0:6]:
    r=requests.get(link)
    soup=BeautifulSoup(r.content , 'html.parser')
    t = soup.find('div', class_='ac-article-content')

    #Get the paragraphs just under the div
    children = t.findChildren("p" , recursive=False)
    art_content_yveline= ""
    for e in children:
        if e.name == 'p':
            art_content_yveline += e.text
    art_content_yveline = art_content_yveline.replace(u'\xa0', u' ')

    #Get the publishing time
    pub_date_yveline= soup.find('div', class_='ac-article-date').find('time')['datetime']
    
    Publish_Dates_yveline.append(pub_date_yveline)
    Articles_Content_yveline.append(art_content_yveline)

date_yc = [datetime.datetime.fromisoformat(x).strftime("%B %d, %Y at %H:%M") for x in Publish_Dates_yveline]

#Recuperation images des articles:
imagesActualiteYvelineCor = [BeautifulSoup(requests.get(li).content, 'html.parser').find('article', {'class': 'js-article-inner'}).find('img')['src'] for li in links_yveline[:6]]
liensActualiteYvelineCor = [li for li in links_yveline[:6]]

In [ ]:
yveline_cor=[]
for i in Articles_Content_yveline:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  yveline_cor.append(paraphrased_result_cleaned)

In [ ]:
Articles_yveline= {
        'dates': date_yc,
        'image actualite': imagesActualiteYvelineCor,
        'actualites' : yveline_cor,
        'liensActualite': liensActualiteYvelineCor
        } 
yveline_df = pd.DataFrame(Articles_yveline)
yveline_df

In [ ]:
yveline_c=yveline_df.transpose()

### Département de Oise 

In [ ]:

liens=[]
ds=[]
link='https://france3-regions.francetvinfo.fr/hauts-de-france/oise'
p=requests.get(link)
soup=BeautifulSoup(p.content , 'html.parser')
for i in soup.findAll('div',{'class':'article-card__date'}):
    ds.append(i['data-date'])
for i in soup.findAll('a',{'class':'article-card__title'}):
    liens.append(i['href'])
li1=[]
li2=[]
lif=[]
for i in  range(len(liens)) :
  if  'coronavirus'  not in liens[i]:
    li1.append(liens[i])
for i in  range(len(li1)) :
  if  'covid'  not in li1[i]:
    li2.append(li1[i])
for i in  range(len(li2)) :
  if  'vaccination'  not in li2[i]:
    lif.append(li2[i])
act=[]
lo=lif[0:6]
for i in lo :
    p=requests.get(i)
    soup=BeautifulSoup(p.content , 'html.parser')
    for i in soup.findAll('div',{'class':'article__chapo'}):
        act.append(i.text)
    for i in soup.findAll('div',{'class':'article__body'}):
        act.append(i.text)   

test1=[]
test2=[]
for i in range(0,len(act),2):
  m=i
  test1.append(act[m])
for i in range(1,len(act),2):
  m=i
  test2.append(act[m])
articleo=[]
for x,y in zip(test1,test2):
  articleo.append(x+y)

articleof=[]
for i in articleo :
  i = re.sub(r'\[[0-9]*\]', ' ', i)
  i = re.sub(r'\s+', ' ', i)
  articleof.append(i.strip())
do=ds[0:6]


#Recuperation images des articles:
imagesActualiteOise = [BeautifulSoup(requests.get(li).content, 'html.parser').find('div', {'class': 'article__main-media'}).find('img')['src'] for li in lif[0:6]]
LiensActualiteOise = [li for li in lif[0:6]]

In [ ]:
oise_tour=[]
for i in articleof:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  oise_tour.append(paraphrased_result_cleaned)

In [ ]:
dict = {'dates': do,
        'image actualite': imagesActualiteOise,
        'acctualites' : oise_tour,
        'liensActualite': LiensActualiteOise
        } 
dfo = pd.DataFrame(dict)
dfo

In [ ]:
oise_t=dfo.transpose()

### Département de Oise (coronavirus)

In [ ]:

req_Oise = requests.get('https://actu.fr/oise/societe/coronavirus')
soup_Oise = BeautifulSoup(req_Oise.content , 'html.parser')
links_Oise = []
artPrevLarge_Oise = soup_Oise.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links_Oise.append(artPrevLarge_Oise)

artPrevSmall_Oise = soup_Oise.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall_Oise:
    links_Oise.append(e.find('a')['href'])

artPrevMedium_Oise= soup_Oise.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium_Oise:
    links_Oise.append(e.find('a')['href'])

Publish_Dates_Oise = []
Articles_Content_Oise = []
lieno=links_Oise[0:6]

for link in lieno:
    r=requests.get(link)
    soup_Oise=BeautifulSoup(r.content , 'html.parser')
    t = soup_Oise.find('div', class_='ac-article-content')

    children = t.findChildren("p" , recursive=False)
    art_content_Oise = ""
    for e in children:
        if e.name == 'p':
            art_content_Oise += e.text
    art_content_Oise = art_content_Oise.replace(u'\xa0', u' ')

    pub_date_Oise = soup_Oise.find('div', class_='ac-article-date').find('time')['datetime']
    
    Publish_Dates_Oise.append(pub_date_Oise)
    Articles_Content_Oise.append(art_content_Oise)


date_oise = [datetime.datetime.fromisoformat(x).strftime("%B %d, %Y at %H:%M") for x in Publish_Dates_Oise]

#Recuperation images des articles:
imagesActualiteOiseCor = [BeautifulSoup(requests.get(li).content, 'html.parser').find('article', {'class': 'js-article-inner'}).find('img')['src'] for li in lieno]
linksActualiteOiseCor = [li for li in lieno]


In [ ]:
oise_articles=Articles_Content_Oise[0:6]
oise_cor=[]
for i in oise_articles:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  oise_cor.append(paraphrased_result_cleaned)

In [ ]:
Articles_Oise = {'dates': date_oise,
                 'image article': imagesActualiteOiseCor,
                 'actualite' : oise_cor,
                 'lienActualite': linksActualiteOiseCor
        } 
Oise_df = pd.DataFrame(Articles_Oise)
Oise_df

In [ ]:
oise_c=Oise_df.transpose()

### Département de Marne

In [ ]:
  # Recuperation des liens des articles
p=requests.get('http://www.marne.fr/')
soup=BeautifulSoup(p.content , 'html.parser')
lim=[]
d=[]
for i in soup.findAll('div',{'class':'slide-data'}):
    dm=i.find('div',{'class':'date'})
    l=i.find('a')
    d.append(dm.text)
    lim.append(l['href'])

datem=d[0:6]
liensm=lim[0:6]
articlesm=[]
for link in liensm:
    p=requests.get(link)
    soup=BeautifulSoup(p.content , 'html.parser')
    for i in soup.findAll('div',{'class':'field field-name-body field-type-text-with-summary apply-editor-styles'}):
      articlesm.append(i.text)
articlemf=[]
for i in articlesm :
    i = re.sub(r'\[[0-9]*\]', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    articlemf.append(i)

#Recuperation images des articles:
imagesActualiteMarne = [BeautifulSoup(requests.get(li).content, 'html.parser').find('div', {'class': 'field field-name-field-visuel field-type-image'}).find('img')['src'] for li in liensm]

In [ ]:
marne_tour=[]
for i in articlemf:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  marne_tour.append(paraphrased_result_cleaned)

In [ ]:
date_marne=[ r.strip(' ').strip().split()[2] for r in datem ]

dict = {'dates': date_marne,
        'image actualite': imagesActualiteMarne,
        'acctualites' : marne_tour[:6]
        } 
dfm = pd.DataFrame(dict)
dfm

In [ ]:
marne_t=dfm.transpose()

### département de Marne (coronavirus)

In [ ]:
  # Recuperation des liens des articles
req_Marne = requests.get('https://actu.fr/la-marne/societe/coronavirus')
soup_Marne = BeautifulSoup(req_Marne.content , 'html.parser')

links_Marne = []
artPrevLarge_Marne = soup_Marne.find('article',class_='ac-preview-article ac-preview-article-large-v').find('a')['href']
links_Marne.append(artPrevLarge_Marne)

artPrevSmall_Marne = soup_Marne.findAll('li',class_='ac-preview-article ac-preview-article-small')
for e in artPrevSmall_Marne:
    links_Marne.append(e.find('a')['href'])

artPrevMedium_Marne= soup_Marne.findAll('li',class_='ac-preview-article ac-preview-article-medium')
for e in artPrevMedium_Marne:
    links_Marne.append(e.find('a')['href'])

  # Recuperation des iimages et contenus des articles
Publish_Dates_Marne = []
Articles_Content_Marne = []

for link in links_Marne[0:6]:
    r=requests.get(link)
    soup_Marne=BeautifulSoup(r.content , 'html.parser')
    t = soup_Marne.find('div', class_='ac-article-content')

    children = t.findChildren("p" , recursive=False)
    art_content_Marne = ""
    for e in children:
        if e.name == 'p':
            art_content_Marne += e.text
    art_content_Marne = art_content_Marne.replace(u'\xa0', u' ')

    pub_date_Marne = soup_Marne.find('div', class_='ac-article-date').find('time')['datetime']
    
    Publish_Dates_Marne.append(pub_date_Marne)
    Articles_Content_Marne.append(art_content_Marne)


#Recuperation images des articles:
imagesActualiteMarneCor = [BeautifulSoup(requests.get(li).content, 'html.parser').find('article', {'class': 'js-article-inner'}).find('img')['src'] for li in links_Marne[0:6]]
linksActualiteMarneCor = [li for li in links_Marne[0:6]]


In [ ]:
imagesActualiteMarneCor

In [ ]:
marne_cor=[]
for i in Articles_Content_Marne:
  text = i
  splitter = SentenceSplitter(language='fr')
  sentence_list = splitter.split(text)

  translated_sentences = [GoogleTranslator(target='en').translate(element) for element in sentence_list]
  paraphrased_sentences = [get_response(e,1) for e in translated_sentences]
  cleaned_paraphrased_sentences = [e[0] for e in paraphrased_sentences]
  sentences_to_french = [GoogleTranslator(target='fr').translate(e) for e in cleaned_paraphrased_sentences]
  paraphrased_result = [' '.join(x for x in sentences_to_french)]
  paraphrased_result_cleaned = str(paraphrased_result).strip('[]').strip("'").strip('"')
  marne_cor.append(paraphrased_result_cleaned)

In [ ]:
date_m = [datetime.datetime.fromisoformat(x).strftime("%B %d, %Y at %H:%M") for x in Publish_Dates_Marne]

Articles_Marne = {'dates': date_m,
        'imageActualite': imagesActualiteMarneCor,
        'actualite' : marne_cor,
        'lienActualite': linksActualiteMarneCor
        } 
Marne_df = pd.DataFrame(Articles_Marne)
Marne_df

In [ ]:
marne_c=Marne_df.transpose()

In [ ]:
acctualites_touristiques = [paris_t,seine_t,yveline_t,oise_t,marne_t]
actualites_sanitaires= [paris_c,seine_c,yveline_c,oise_c,marne_c]
dict = {'actualites_touristiques': acctualites_touristiques,
        'actualites_sanitaires': actualites_sanitaires 
        } 
    
df = pd.DataFrame(dict,index =['paris', 'seine_marne','yveline','oise','marne'])
    
df 


In [ ]:
with open('Acctualite_touristiques_sanitaires.json', 'w', encoding='utf-8') as file:
    df.to_json(file, force_ascii=False)

In [ ]:
from google.colab import files
files.download('Acctualite_touristiques_sanitaires.json')